In [1]:
!pip install tflearn 

In [11]:
import nltk
import random
import pickle
import json
import numpy as np
import warnings

import tensorflow as tf
import tflearn
from nltk.stem.lancaster import LancasterStemmer

warnings.filterwarnings('ignore')

In [2]:
''' reading json file '''
with open('intents.json', 'rb') as json_file:
    ''' load data '''
    data = json.load(json_file)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [4]:
wor = []
cls = []
doc= []
ig = ['?']

for i in data['intents']:
    for p in i['patterns']:
        ''' converting p into tokens '''
        w = nltk.word_tokenize(p)
        
        '''add word to the words list'''
        wor.extend(w)
        
        '''add word(s) to documents'''
        doc.append((w, i['tag']))
        
        '''add tags to our classes list'''
        if i['tag'] not in cls:
            cls.append(i['tag'])

In [5]:
''' text cleaning '''

''' stemmer '''
stem = LancasterStemmer()

'''applying stemming '''
w = [stem.stem(w.lower()) for w in wor if w not in ig]

''' sorting words '''
w = sorted(list(set(w)))

In [6]:
w[:10]

["'s",
 'acceiv',
 'address',
 'anyon',
 'ar',
 'bye',
 'can',
 'card',
 'cash',
 'cours']

In [7]:
''' creating training data and training label i.e. X and y'''
X = []
y = []

'''create an empty array for output'''
out_emp = [0] * len(cls)

for d in doc:
    '''initialize bag of words'''
    b = []
    
    '''list of tokenized words for the pattern'''
    pw = d[0]
    
    ''' stemming '''
    pw = [stem.stem(word.lower()) for word in pw]
    
    '''creating bag of words array'''
    for w in wor:
        b.append(1) if w in pw else b.append(0)

    ''' output is '1' for current tag and '0' for rest of other tags '''
    out_r = list(out_emp)
    out_r[cls.index(d[1])] = 1

    X.append([b, out_emp])

''' shuffling '''
random.shuffle(X)

''' converting training data into numpy array '''
X = np.array(X)

In [8]:
X_ = list(X[:,0])
y = list(X[:,1])

In [9]:
X_[0][:5]

[0, 0, 0, 0, 0]

In [10]:
''' Neural mwork Model '''
m = tflearn.input_data(shape=[None, len(X_[0])])
m = tflearn.fully_connected(m, 10)
m = tflearn.fully_connected(m, 10)
m = tflearn.fully_connected(m, len(y[0]), activation='softmax')
m = tflearn.regression(m)

''' Defining model and setting up tensorboard'''
model = tflearn.DNN(m, tensorboard_dir='tflearn_logs')

''' training '''
model.fit(X_, y, n_epoch=500, batch_size=8, show_metric=True)

Training Step: 1999  | time: 0.036s
| Adam | epoch: 500 | loss: 0.00000 - acc: 0.1970 -- iter: 24/31
Training Step: 2000  | time: 0.048s
| Adam | epoch: 500 | loss: 0.00000 - acc: 0.1916 -- iter: 31/31
--


In [16]:
pickle.dump( {'words':wor, 'classes':cls, 'X':X_, 'y':y}, open( "training_data", "wb" ) )

In [18]:
'''restoring all the data structures'''
d = pickle.load( open( "training_data", "rb" ) )
d_w = d['words']
d_c = d['classes']
X_ = d['X']
y = d['y']

In [23]:
def clean_sent(sent):
    ''' convert sent into tokens '''
    sent_word = nltk.word_tokenize(sent)
    'stemming'''
    sent_word = [stem.stem(word.lower()) for word in sent_word]
    return sent_word

def bag_of_words(sent, w, sd=False):
    ''' calling clean_sent function '''
    sent_word = clean_sent(sent)
    '''generating bag of words'''
    bow = [0]*len(w)  
    for s in sent_word:
        for i, w_ in enumerate(w):
            if w_ == s: 
                bow[i] = 1
                if sd:
                    print ("found in bag: %s" % w_)

    return(np.array(bow))

In [46]:
ERROR_THRESHOLD = 0.10


def classification(sent):
    ''' prediction '''
    results = model.predict([bag_of_words(sent, d_w)])[0]
    
    '''filter out predictions below a threshold'''
    res = [[i,r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    
    '''sorting'''
    res.sort(key=lambda x: x[1], reverse=True)
    ret_l = []
    for r in res:
        ret_l.append((d_c[r[0]], r[1]))
    
    ''' return tuple of intent and probability '''
    return ret_l

def resp(sent, userID='123', s_d=False):
    res = classification(sent)
    '''if we have a classification then find the matching intent tag'''
    if res:
        while res:
            for i in data['intents']:
                
                '''find a tag matching the first resul'''
                if i['tag'] == res[0][0]:
                    
                    '''a random response from the intent'''
                    return print(random.choice(i['responses']))

            res.pop(0)

In [47]:
classification('What are you hours of operation?')

[('location', 0.11119011),
 ('goodbye', 0.111175604),
 ('payments', 0.11114356),
 ('greeting', 0.111143544),
 ('hours', 0.11108659),
 ('thanks', 0.111074775),
 ('deliveryoption', 0.111074075),
 ('menu', 0.11107163),
 ('todaysmenu', 0.11104013)]

In [48]:
resp('What are you hours of operation?')

We are on the intersection of London Alley and Bridge Avenue.


In [50]:
resp('What is menu for today?')

Our Address is: 1000 Bridge Avenue, London EC3N 4AJ, UK


In [51]:
resp('Do you accept Credit Card?')

We are on the intersection of London Alley and Bridge Avenue.


In [52]:
resp('Where can we locate you?')

We are on the intersection of London Alley and Bridge Avenue.


In [54]:
resp('Bye')

Good to see you again


In [58]:
ERROR_THRESHOLD = 0.05


def classification(sent):
    ''' prediction '''
    results = model.predict([bag_of_words(sent, d_w)])[0]
    
    '''filter out predictions below a threshold'''
    res = [[i,r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    
    '''sorting'''
    res.sort(key=lambda x: x[1], reverse=True)
    ret_l = []
    for r in res:
        ret_l.append((d_c[r[0]], r[1]))
    
    ''' return tuple of intent and probability '''
    return ret_l

def resp(sent, userID='123', s_d=False):
    res = classification(sent)
    '''if we have a classification then find the matching intent tag'''
    if res:
        while res:
            for i in data['intents']:
                
                '''find a tag matching the first resul'''
                if i['tag'] == res[0][0]:
                    
                    '''a random response from the intent'''
                    return print(random.choice(i['responses']))

            res.pop(0)

In [59]:
resp('Can you please let me know the delivery options?')

We are on the intersection of London Alley and Bridge Avenue.


In [60]:
resp('What is menu for today?')

We are situated at the intersection of London Alley and Bridge Avenue


In [63]:
resp("Hi there!", s_d=True)

Hello, thanks for visiting


In [64]:
resp('What is menu for today?')

We are situated at the intersection of London Alley and Bridge Avenue
